In [2]:
import geopandas as gpd
import rasterio
import rasterstats
import pandas as pd
from pathlib import Path
from datetime import datetime

# === Paths ===
data_dir = Path(r"F:\School\Jupyter Stuff\Data Science Projects\climate-bee-population-model\data\Vegetation Data")
shapefile = data_dir / "cb_2018_us_state_20m.shp"

# === Load contiguous US shapefile ===
states = gpd.read_file(shapefile)[["NAME", "geometry"]]
states.rename(columns={"NAME": "State"}, inplace=True)
excluded = ["Hawaii", "Alaska", "Puerto Rico", "Guam", "American Samoa", "Northern Mariana Islands", "Virgin Islands"]
states = states[~states["State"].isin(excluded)].reset_index(drop=True)

# === Collect all NDVI raster files ===
tif_files = sorted(data_dir.glob("VHP.G04*.SM.SMN.tif"))

records = []

# === Process rasters ===
for tif in tif_files:
    name = tif.name
    try:
        code = name.split(".P")[1][:7]
        year = int(code[:4])
        week = int(code[4:])
        date = datetime.strptime(f"{year}{week}-1", "%Y%W-%w")

        with rasterio.open(tif) as src:
            stats = rasterstats.zonal_stats(
                states,
                src.read(1),
                affine=src.transform,
                stats=["mean"],
                nodata=src.nodata,
                all_touched=True,
            )

        for s, st in zip(states["State"], stats):
            if st["mean"] is not None:
                records.append((s, date, st["mean"]))
    except Exception:
        pass  # completely silent

# === Build and save DataFrame ===
df = pd.DataFrame(records, columns=["State", "Date", "Mean_NDVI"])
df["YearMonth"] = df["Date"].dt.to_period("M")

monthly = (
    df.groupby(["State", "YearMonth"])["Mean_NDVI"]
    .mean()
    .reset_index()
)

monthly["Date"] = monthly["YearMonth"].dt.to_timestamp()
monthly = monthly[["State", "Date", "Mean_NDVI"]].sort_values(["State", "Date"])

out_csv = data_dir / "NDVI_State_Monthly_1985_2025.csv"
monthly.to_csv(out_csv, index=False)

✅ Loaded 49 contiguous US states
🛰️ Found 2071 NDVI rasters


S:\Anaconda\Lib\site-packages\rasterstats\io.py:335: NodataWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn(


📅 Processed VHP.G04.C07.j01.P2019001.SM.SMN.tif → 2019-01-07
📅 Processed VHP.G04.C07.j01.P2019002.SM.SMN.tif → 2019-01-14
📅 Processed VHP.G04.C07.j01.P2019003.SM.SMN.tif → 2019-01-21
📅 Processed VHP.G04.C07.j01.P2019004.SM.SMN.tif → 2019-01-28
📅 Processed VHP.G04.C07.j01.P2019005.SM.SMN.tif → 2019-02-04
📅 Processed VHP.G04.C07.j01.P2019006.SM.SMN.tif → 2019-02-11
📅 Processed VHP.G04.C07.j01.P2019007.SM.SMN.tif → 2019-02-18
📅 Processed VHP.G04.C07.j01.P2019008.SM.SMN.tif → 2019-02-25
📅 Processed VHP.G04.C07.j01.P2019009.SM.SMN.tif → 2019-03-04
📅 Processed VHP.G04.C07.j01.P2019010.SM.SMN.tif → 2019-03-11
📅 Processed VHP.G04.C07.j01.P2019011.SM.SMN.tif → 2019-03-18
📅 Processed VHP.G04.C07.j01.P2019012.SM.SMN.tif → 2019-03-25
📅 Processed VHP.G04.C07.j01.P2019013.SM.SMN.tif → 2019-04-01
📅 Processed VHP.G04.C07.j01.P2019014.SM.SMN.tif → 2019-04-08
📅 Processed VHP.G04.C07.j01.P2019015.SM.SMN.tif → 2019-04-15
📅 Processed VHP.G04.C07.j01.P2019016.SM.SMN.tif → 2019-04-22
📅 Processed VHP.G04.C07.

In [1]:
import tarfile
from pathlib import Path

data_dir = Path(r"F:\School\Jupyter Stuff\Data Science Projects\climate-bee-population-model\data\Vegetation Data")
archives = sorted(data_dir.glob("VHP*.tar.gz"))

for archive_path in archives:
    with tarfile.open(archive_path, "r:gz") as tar:
        members = [m for m in tar.getmembers() if m.name.endswith(".SM.SMN.tif")]
        if members:
            tar.extractall(path=data_dir, members=members, filter="data")

🗂️  Found 39 archives to extract.
📦 Extracting 44 NDVI files from VHP.G04.C07_1985.tar.gz ...


C:\Users\Stor\AppData\Local\Temp\ipykernel_25064\2533170425.py:21: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path=data_dir, members=members)


📦 Extracting 52 NDVI files from VHP.G04.C07_1986.tar.gz ...
📦 Extracting 52 NDVI files from VHP.G04.C07_1987.tar.gz ...
📦 Extracting 45 NDVI files from VHP.G04.C07_1988.tar.gz ...
📦 Extracting 52 NDVI files from VHP.G04.C07_1989.tar.gz ...
📦 Extracting 52 NDVI files from VHP.G04.C07_1990.tar.gz ...
📦 Extracting 52 NDVI files from VHP.G04.C07_1991.tar.gz ...
📦 Extracting 52 NDVI files from VHP.G04.C07_1992.tar.gz ...
📦 Extracting 52 NDVI files from VHP.G04.C07_1993.tar.gz ...
📦 Extracting 36 NDVI files from VHP.G04.C07_1994.tar.gz ...
📦 Extracting 49 NDVI files from VHP.G04.C07_1995.tar.gz ...
📦 Extracting 52 NDVI files from VHP.G04.C07_1996.tar.gz ...
📦 Extracting 52 NDVI files from VHP.G04.C07_1997.tar.gz ...
📦 Extracting 52 NDVI files from VHP.G04.C07_1998.tar.gz ...
📦 Extracting 52 NDVI files from VHP.G04.C07_1999.tar.gz ...
📦 Extracting 52 NDVI files from VHP.G04.C07_2000.tar.gz ...
📦 Extracting 52 NDVI files from VHP.G04.C07_2001.tar.gz ...
📦 Extracting 52 NDVI files from VHP.G04.

In [3]:
data_dir = Path(r"F:\School\Jupyter Stuff\Data Science Projects\climate-bee-population-model\data\Vegetation Data")
csv_path = data_dir / "NDVI_State_Monthly_1985_2025.csv"

df = pd.read_csv(csv_path)
df["Mean_NDVI"] = df["Mean_NDVI"] / 1000.0
df["Mean_NDVI"] = df["Mean_NDVI"].clip(-1, 1)

fixed_csv = data_dir / "NDVI_State_Monthly_1985_2025_FIXED.csv"
df.to_csv(fixed_csv, index=False)

✅ Fixed scaling and saved to: F:\School\Jupyter Stuff\Data Science Projects\climate-bee-population-model\data\Vegetation Data\NDVI_State_Monthly_1985_2025_FIXED.csv
     State        Date  Mean_NDVI
0  Alabama  1985-03-01  -0.038867
1  Alabama  1985-04-01  -0.038774
2  Alabama  1985-05-01  -0.038693
3  Alabama  1985-06-01  -0.038695
4  Alabama  1985-07-01  -0.038741
